크롤링

In [2]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep
#크롬 드라이버 최신 버전 https://googlechromelabs.github.io/chrome-for-testing/

In [146]:
##최종##
store_name_lst = []
category_lst = []
phone_num_lst = []
address_lst = []

def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    #가게명 클릭 후 우측 프레임 로딩 대기
    WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.ID, "entryIframe"))
    )
    sleep(0.1) #혹시 몰라 추가시간

    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)


options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
#driver = webdriver.Chrome(ChromeDriverManager().install()) # 크롬 드라이버 설치하지 않은 경우
driver = webdriver.Chrome(options=options) # 크롬 드라이버 설치한 경우
 
# 대기 시간
driver.implicitly_wait(time_to_wait=3)
 
# 반복 종료 조건
loop = True

driver.get("https://map.naver.com/v5/") #네이버지도 열기
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

keyword = '대전 중구 태권도장'

search_box = driver.find_element(By.CLASS_NAME,"input_search") #검색창 찾기
search_box.send_keys(keyword) #검색어 입력
search_box.send_keys(Keys.ENTER) #검색

 
driver.implicitly_wait(5)

while(loop):
    switch_left()

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)


    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 1500;", scrollable_element)
 
        # 페이지 로드를 기다림
        sleep(0.5)  # 동적 콘텐츠 로드 시간에 따라 조절
 
        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
 
        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break
 
        last_height = new_height
 

    ############## 현재 page number 가져오기 ##############
 
    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text
    
    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함

    elements = driver.find_elements(By.XPATH,'//*[@id = "_pcmap_list_scroll_container"]/ul/li') #광고 포함 페이지 전체 elements

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elements)) + '\033[0m' + '개(광고 포함)의 가게를 찾았습니다.\n')

    '''
    for index, e in enumerate(elemets, start=1):
        try:#광고인 경우
            ad = e.find_element(By.CLASS_NAME,'gU6bV').find_element(By.XPATH, ".//span") 
            continue #다음 for문 루프
        except:
            ####final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span[1]")
            elemets.append(e) #논광고만 삽입
            
        #####print(str(index) + ". " + final_element.text)


    '''
    switch_left()
 
    
    for index, e in enumerate(elements, start=1): #기존: elemets
        store_name = '' # 가게 이름
        category = '' # 카테고리
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호
 
        switch_left()
        '''가게명 클릭 '''   
        #우측 그림 or 없음 - type1
        if e.get_attribute("data-laim-exp-id")=="undefined": 
            e.find_elements(By.XPATH, "./div[1]/div")[-1].find_element(By.XPATH,"./a[1]/div/div/span[1]").click() 

        #아래에 그림있는 경우 - type2
        elif e.get_attribute("data-laim-exp-id")=="undefinedundefined":
            e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, "./a/div/div/span[1]").click() 
              
        #광고 - type3
        else:  #"undefined*e"
            continue #다음 for문 루프로
        ''''''
        # button = driver.find_element_by_xpath(".//a/div/div/span")
        # driver.execute_script("arguments[0].click();", button)


        switch_right()
 
        ##### 상세정보 크롤링 ####
        title = driver.find_element(By.XPATH,'//div[@class = "zD5Nm undefined"]')
        # store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')
 
        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
        # 카테고리
        category = title.find_elements(By.XPATH,'.//div[1]/div[1]/span')[-1].text
        #전화번호
        try:
            phone_num = driver.find_element(By.XPATH,'//span[@class = "xlx7Q"]').text
            driver.implicitly_wait(0.5) #전화번호를 0.5초 안에 못찾으면, 원래 없다고 간주
        except:
            phone_num = '-'
  
        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class = "LDgIH"]').text

        #리뷰 수
        
        try:
            reviews = title.find_elements(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot"]') #[방문자, 블로그]
            visitor_review =[review.find_element(By.XPATH,'./a').text for review in reviews]
        

            driver.implicitly_wait(0.1) #0.1초 안에 못찾으면, 없는 것으로 간주
        except:
            visitor_review = []


        #별점
        try:

            score = title.find_element(By.XPATH, './/div[@class = "dAsGb"]/span[@class = "PXMot LXIwF"]').text

            driver.implicitly_wait(0.1) #0.1초 안에 못찾으면, 없는 것으로 간주
        except:
            score = '-' 


        print(f'가게명: {store_name},   업종: {category},   전화번호: {phone_num},   주소: {address},   별점: {score}   리뷰: {visitor_review}')
        store_name_lst.append(store_name)
        category_lst.append(category)
        phone_num_lst.append(phone_num)
        address_lst.append(address)
    
    switch_left()

    # 이건 페이지 넘어갈때마다 확인 -> true라면 마지막 페이지임
    next_page = driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1]

    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page.get_attribute('aria-disabled') == 'false'):
        driver.find_elements(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a')[-1].click()
    # 아닐 경우 루프 정지
    else:
        loop = False #모두 끝

Exception ignored in: <function Service.__del__ at 0x00000212000718A0>
Traceback (most recent call last):
  File "C:\Users\jiho\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "C:\Users\jiho\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\jiho\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 131, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jiho\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 120, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jiho\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\utils.py", line 101, in is_connectable
    socket

현재 1 페이지 / 총 50개(광고 포함)의 가게를 찾았습니다.

type 1
가게명: MTA그린태권도,   업종: 태권도장,   전화번호: 0507-1476-2339,   주소: 대전 중구 평촌로 84 극동상가 306호,   별점: 별점
4.92   리뷰: ['방문자리뷰 113', '블로그리뷰 36']
type 1
가게명: 버드내태권도,   업종: 태권도,   전화번호: 010-4456-5532,   주소: 대전 중구 태평로 18 4층,   별점: -   리뷰: ['방문자리뷰 12']
type 1
가게명: 용인대온누리태권도,   업종: 태권도,   전화번호: 0507-1438-1181,   주소: 대전 중구 태평로 15,   별점: 별점
4.62   리뷰: ['방문자리뷰 84', '블로그리뷰 1']
type 1
가게명: 용인대다정태권도,   업종: 태권도,   전화번호: 042-222-3001,   주소: 대전 중구 목동로 37 목양마을아파트 단지상가 3층,   별점: -   리뷰: ['방문자리뷰 22', '블로그리뷰 7']
type 1
가게명: 해오름태권도장,   업종: 태권도,   전화번호: 0507-1379-2712,   주소: 대전 중구 대종로 205 2층,   별점: -   리뷰: ['방문자리뷰 37', '블로그리뷰 1']
type 1
가게명: 경희태권도,   업종: 태권도,   전화번호: 042-581-0177,   주소: 대전 중구 보문산로 31 114동 후문상가 3층,   별점: -   리뷰: ['방문자리뷰 28']
type 1
가게명: 용인대우리태권도 목동관,   업종: 태권도,   전화번호: 042-252-2002,   주소: 대전 중구 목동로 42 대아플러스,   별점: -   리뷰: ['방문자리뷰 4']
type 1
가게명: 스포렉스태권도,   업종: 태권도,   전화번호: 042-535-3500,   주소: 대전 중구 유등천동로 468-2 2층 스포렉스태권도,   별점: -   리뷰: ['방문자리뷰 20']
type 1
가게명: 꾸러기